# VGG-16 Implementation Using PyTorch
In this notebook, we will implement the VGG-16 model using PyTorch. 

In [ ]:
# Required Libraries
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable
import pandas as pd;
from sklearn.model_selection import train_test_split
import numpy as np;


## Define the VGG-16 Neural Network
Here we will define the VGG-16 model.


In [ ]:
class VGG16(nn.Module):
    def __init__(self, num_classes=1000):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


## Upload your dataset from Roboflow
Here we are implementing an in-code upload of the dataset from roboflow.com. Please ensure that the dataset is in a format compatible with PyTorch (e.g., ImageFolder structure).


In [ ]:
# Install roboflow
!pip install roboflow

# Replace with your roboflow dataset link
dataset_link = "roboflow:YOUR_ROBOFLOW_LINK_HERE"

# Importing the dataset
from roboflow import Roboflow

rf = Roboflow(api_key="YOUR_ROBOFLOW_API_KEY_HERE")
dataset = rf.dataset(dataset_link)

# Unpack the dataset
dataset.unpack("path_to_save_data")


## Preparing the DataLoader
We now need to prepare the DataLoader for our training and testing sets. Please adjust the batch_size based on the memory of your GPU.


In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Define your transforms
transform = transforms.Compose([transforms.Resize((224,224)), 
                                transforms.ToTensor(), 
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Create your dataloaders
dataset = ImageFolder('path_to_save_data', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Splitting the dataset into training and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)


## Train the Model
We are now ready to train our model. Here we will be using the Adam optimizer and Cross Entropy Loss. Feel free to tweak these parameters according to your dataset and preferences.


In [ ]:
# Instantiate the model
vgg16 = VGG16(num_classes=len(dataset.classes)).to('cuda')

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg16.parameters(), lr=0.001)

# Training the model
num_epochs = 10
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())
        
        # Forward pass
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item()}**Cell 11: Markdown**



# Test the Model
After training, we can now test the model with the test set. It's important to remember that we should set the model to 'eval' mode when testing or validating, as this will turn off certain layers that behave differently during training mode, such as dropout layers.

In [ ]:
# Test the model
vgg16.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())
        outputs = vgg16(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print(f'Test Accuracy of the model on the test images: {100 * correct / total}')

## Saving the Model
Finally, if you're satisfied with your model performance, you can save it for future use.

In [ ]:
# Save the model checkpoint
torch.save(vgg16.state_dict(), 'vgg16.ckpt')

## Loading the Model
To load the model, just use the `load_state_dict` method, like so:

In [ ]:
# Loading the model
model = VGG16(num_classes=len(dataset.classes)).to('cuda')
model.load_state_dict(torch.load('vgg16.ckpt'))

Treinado em: 